In [25]:
import yfinance as yf
from haystack.nodes import LinkContentFetcher
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import  DensePassageRetriever, AnswerParser
from haystack.nodes.prompt import PromptTemplate
import os
from haystack.nodes import PromptNode
from haystack import Pipeline
from haystack.nodes import BM25Retriever


In [26]:
# Model configurations
hfAPIKey = os.getenv("HF_API_KEY")
hfModelName = os.getenv("HF_MODEL_NAME")

In [27]:
gold = yf.Ticker("GC=F")
news = gold.news


In [28]:
links = []

for new in news:
    links.append(new["link"])

print(links)

['https://finance.yahoo.com/m/a19af4a1-5c56-37d7-b5fb-ec7ec3b6ecca/gold-edges-higher-amid.html', 'https://finance.yahoo.com/news/1-newmont-shareholders-vote-favor-225344267.html', 'https://finance.yahoo.com/news/newmont-shareholders-vote-favor-newcrest-223053608.html', 'https://finance.yahoo.com/news/global-markets-bond-yields-ease-210650890.html', 'https://finance.yahoo.com/news/bitcoin-sinks-nearly-3-26-210130103.html', 'https://finance.yahoo.com/news/closing-prices-crude-oil-gold-192049208.html', 'https://finance.yahoo.com/news/barrick-gold-explore-discoveries-loulo-190200225.html', 'https://finance.yahoo.com/news/jim-cramer-doesnt-know-bitcoin-183130946.html']


In [29]:
lcf = LinkContentFetcher()

docs = []

for link in links:
    docs.append(lcf.fetch(link)[0])



In [31]:
# faissDocumentStore = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# retriever = DensePassageRetriever(
#             document_store=faissDocumentStore,
#             query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#             passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#             use_gpu=True,
#             embed_title=True,
#     )

# faissDocumentStore.delete_documents()
# faissDocumentStore.write_documents(docs)

# faissDocumentStore.update_embeddings(retriever=retriever)


Updating BM25 representation...: 100%|██████████| 8/8 [00:00<00:00, 1902.93 docs/s]


In [32]:
promptTemplate = PromptTemplate(prompt = """"Answer the following query based on the provided context. If the context does
                                                not include an answer, reply with 'The data does not contain information related to the question'.\n
                                                Query: {query}\n
                                                Documents: {join(documents)}
                                                Answer: 
                                            """,
                                            output_parser=AnswerParser())

# Prompt Node initialization
promptNode = PromptNode(hfModelName,api_key=hfAPIKey,max_length=256,model_kwargs={"stream":True})


In [33]:
# Pipeline
pipeline = Pipeline()
pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipeline.add_node(component=promptNode, name="PromptNode", inputs=["Retriever"])

In [34]:
output = pipeline.run(query="What can you tell me about gold?")

print(output)

{'results': [], 'invocation_context': {'query': 'What can you tell me about gold?', 'documents': [<Document: {'content': "Jim Cramer Doesn't Know Bitcoin\nRead full article\nOctober 11, 2023 at 11:31 AM\n·3 min read\n“I can’t go out with bitcoin because I can’t be in something where Mr. Bitcoin is about to go down big.”\nSo said Jim Cramer today live on national TV.\nThis is an excerpt from\xa0The Node newsletter, a daily roundup of the most pivotal crypto news on CoinDesk and beyond. You can subscribe to get the full newsletter here .\nThe veteran blow-hard, former trader and CNBC host was talking about hedge investments at a time of inflation and economic turmoil. He told his audience that they would be better putting money into gold, the traditional safe-haven, rather than crypto. “The charts…suggest you need to ignore the crypto cheerleaders now that bitcoin’s bouncing. And if you seriously want a real hedge against inflation or economic chaos, she says you should stick with gold,”